# ANALYSIS OF FILMS AT THE BOX OFFICE

## OVERVIEW

This project uses exploratory data analysis to generate insights in movie patterns. The goal is to determine what type of films are currently doing best
at the box office for the company invest in as it starts its new movie studio. By cleaning, merging, grouping the data we understand what films
are apprppriate and how they are created.

## BUSINESS PROBLEM

The company is determined to launch a new movie studio though they lack insight in creating movies. The aim of this analysis is to determine
what type of films are currently doing best at the box office, then turn the findings into practical recommendations to help the company decide
what type of films to create.

## DATA UNDERSTANDING

This project uses movie datasets from multiple locations with different file formats, CSV, TSV, SQLite.
The primary dataset being used is the 
1. IMDB data which has a formart of SQLite Database, with **movie_basics** and **movie_ratings** as the relevant tables.

movie_basics table contain columns such as 
* movie_id "unique movie id number"
* primary_title "movie title displayed for audience"
* original_title "movie title as the createrors intended it"
* start_year "year movie started"
* runtime_minutes "movie duration in minutes"
* genres "movie categories"
    
and the movie_ratings table contain columns such as 
    
* movie_id "unique movie id number"
* averagerating "IMDb user ratings"
* numvotes- "number of user votes per movie"

**From the IMDB data we have "movie_id" column as the unique identification for movie_basics and movie_ratings tables**

2. Box Office Mojo Data which has a formart of csv (bom.movie_gross.csv.gz), it includes the following columns
* title "representing the movie title"
* studio "representing the production studio"
* domestic_gross "total revenue generated by films domestically"
* foreign_gross "total revenue generated by films internationaly"
* year "year of movie release"



# DATA PREPARATION

In [30]:
import pandas as pd
import sqlite3
import pandasql

In [31]:
movie_gross_df = pd.read_csv("./zippedData/bom.movie_gross.csv.gz")
movie_gross_df.head(7)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
5,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010
6,Iron Man 2,Par.,312400000.0,311500000,2010


In [3]:
# creating a connection to sqlite database, then list all table names in the db
conn=sqlite3.connect("./zippedData/im.db")
df=pd.read_sql(
"""
SELECT name
FROM sqlite_master
WHERE type = "table"

""", conn)
df

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [4]:
movie_basics_df = pd.read_sql("""
SELECT *
FROM movie_basics
""", conn)
movie_basics_df.head(7)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
5,tt0111414,A Thin Life,A Thin Life,2018,75.0,Comedy
6,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller"


In [5]:
movie_ratings_df = pd.read_sql("""
SELECT *
FROM movie_ratings
""", conn)
movie_ratings_df.head(7)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
5,tt1069246,6.2,326
6,tt1094666,7.0,1613


In [6]:
movie_details_df = pd.read_sql("""
SELECT *
FROM movie_basics
JOIN movie_ratings
USING (movie_id);
""", conn)
movie_details_df.head(7)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119
5,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller",4.1,32
6,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",8.1,263


In [ ]:
merged_df = pd.read_sql("""
SELECT *
FROM movie_gross_df
JOIN movie_ratics_df
ON movie_ratics_df.movie_id, movie_gross_df.title


## Data cleaning

Data cleaning in the (*movie_basics* and *movie_ratings*) movie_details table

In [7]:
movie_details_df.shape

(73856, 8)

1. **genres column**

In [8]:
# nall values in the genres column
movie_details_df["genres"].isna().sum()

804

In [9]:
movie_details_df["genres"].mode()

0    Drama
Name: genres, dtype: object

**we fill NaNs with mode because genres is a nominal categorical variable**

In [10]:
movie_details_df["genres"]=movie_details_df["genres"].fillna("Drama")

In [12]:
movie_details_df["genres"].isna().sum()

0

2. **Average ratings** column (movie ratings)

In [13]:
movie_details_df["averagerating"].isna().sum()

0

In [14]:
movie_details_df["numvotes"].isna().sum()

0

3. **runtime_minutes** column (movie duration)** 

In [15]:
movie_details_df["runtime_minutes"].isna().sum()

7620

**We fill NaN with either mode, median or mean in numerical variables**

In [19]:
movie_details_df["runtime_minutes"].mode()

0    90.0
Name: runtime_minutes, dtype: float64

In [20]:
movie_details_df["runtime_minutes"]=movie_details_df["runtime_minutes"].fillna(90.0)

In [21]:
movie_details_df["runtime_minutes"].isna().sum()

0

4. **primary_title** and **original_title** columns

In [23]:
movie_details_df["primary_title"].isna().sum()

0

In [25]:
movie_details_df["original_title"].isna().sum()

0

In [27]:
movie_details_df["start_year"].isna().sum()

0

Data cleaning in *movie_gross* table

1. **studio** column (movie studios)

In [35]:
movie_gross_df["studio"].shape

(3387,)

In [28]:
movie_gross_df["studio"].isna().sum()

5

In [36]:
movie_gross_df["studio"].mode()

0    IFC
Name: studio, dtype: object

In [38]:
movie_gross_df["studio"] = movie_gross_df["studio"].fillna("IFC")

In [39]:
movie_gross_df["studio"].isna().sum()

0

2. **domestic_gross** column (revenue generated domestically)

In [42]:
movie_gross_df["domestic_gross"].isna().sum()

28

In [46]:
movie_gross_df["domestic_gross"].median()

1400000.0

In [47]:
movie_gross_df["domestic_gross"] = movie_gross_df["domestic_gross"].fillna(1400000.0)

In [48]:
movie_gross_df["domestic_gross"].isna().sum()

0

3. **foreign_gross** column (revenue generated internationaly)

In [50]:
movie_gross_df["foreign_gross"].isna().sum()

1350

In [55]:
movie_gross_df["foreign_gross"].mode()

0    1200000
Name: foreign_gross, dtype: object